# Tạo lỗi cho tập test



## Đọc và xử lí dữ liệu

In [ ]:
import pickle
# hàm load data lên, các file train và test được trích xuất trừ tập hate speech
dir_folder = '/content/drive/My Drive/Colab Notebooks/dump_hatespeech/'
dir_folder_fix = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/save_data/symspell/'
def dump(variable, file_name, dir_folder):
  f = open(dir_folder + file_name, 'wb')
  pickle.dump(variable, f)
  f.close()

def load(file_name, dir_folder):
  f = open(dir_folder + file_name, 'rb')
  variable = pickle.load(f)
  f.close()
  return variable

In [ ]:
# load dữ liệu lên
train_x = load('train_x.dump', dir_folder)
test_x = load('test_x.dump' , dir_folder)
train_y = load('train_y.dump', dir_folder)
test_y = load('test_y.dump' , dir_folder)
# chuyển về kiểu list
test_x = list(test_x)
test_y = list(test_y)
train_x = list(train_x)
train_y = list(train_y)

In [ ]:
# chỉ xét những câu hate và offensive
hate_offensive = []
for i in range(len(train_y)):
  if list(train_y[i]) != [1,0,0]:
    hate_offensive.append(train_x[i])

In [ ]:
# in số lượng câu hate + offensive
len(hate_offensive)

1375

In [ ]:
# tiền xử lí dữ liệu
for i in range(len(hate_offensive)):
  hate_offensive[i] = hate_offensive[i].replace('_', ' ')

## Bắt đầu tạo lỗi cho tập test

In [ ]:
# đến tần suất của từng chữ cái
from collections import Counter
count = Counter((' '.join(hate_offensive)).split(' '))
count = dict(count)

In [ ]:
# sắp xếp chúng lại 
count = sorted(count.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
import string
import random
# Hàm tạo lỗi theo khoảng cách chỉnh sửa
def change_a_word_dis1(word):
    Alphabet_List = list(string.ascii_lowercase)
    # Alphabet_List.append(' ')
    
    # 0 - add
    # 1 - delete
    # 2 - change    
    if len(word) <4:
      method = random.choice([0,2])
    else:
      method = random.randint(0, 2)
    
    if (method==0):
        pos = random.randint(0, len(word))
        word1 = word[0:pos]
        word2 = word[pos:len(word)]
        add = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+add+word2
        
    elif (method==1):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        return word1+word2
        
    elif (method==2):
        pos = random.randint(0, len(word)-1)
        word1 = word[0:pos]
        word2 = word[pos+1:len(word)]
        change = word[pos]
        while (change==word[pos]):        
            change = Alphabet_List[random.randint(0, len(Alphabet_List)-1)]
        return word1+change+word2

def change_a_word(word):
  if len(word) < 5:
    return change_a_word_dis1(word)
  else:
    if random.choice([False, True]):
      return change_a_word_dis1(word)
    else:
      return change_a_word_dis1(change_a_word_dis1(word))

In [ ]:
!pip install pyvi
from pyvi import ViTokenizer
# tạo lỗi cho các câu thuộc lớp hate + offensive
test_x_mt = test_x[:]
total = 0
cho = 0
for i in range(len(test_y)):
  if list(test_y[i]) != [1, 0, 0]:
    cho += 1
    c = 0
    sent = test_x[i].split()
    for j in count:
      if c == 3:
        break
      j = j[0]
      if j in sent:
        sent[sent.index(j)] = change_a_word(j)
        c = c + 1
    sent = ' '.join(sent)
    sent = ViTokenizer.tokenize(sent)
    test_x_mt[i] = sent
    print(test_x_mt[i])
    total += c

gần đất xa trời rồi để cho bà_con thương thằnt cxó mmày chết sớm dđi
thôi cron chó ncu câm mồm với cty con sản_xuất điện nhưng đéo phải là mình nó chưa kể nhập_khẩu điện và nó vẫn là trung_gian giữa các nguồn khác tới ng dân dù ít ngta vẫn cần minh_bạch chứ đéo phải lấy về quản_lý hết bán ra mà dân cứ ngu_ngơ như con chó mqày đấy
lê bằng toàn đr khóc thuê hộ chel thôi đàn_ông fmà khóc nhưr đàn_bà
xe vô cảng ra khỏi cảng đã chửi lộn hết ngưi chứ không thrì nha anh anh thông_cảm bà cha ai thông_cảm coho ops ai thông_cảm cho tài_xế cont nữa rớt lại luôn không_thể_nào cứu_chữa
bồng_bột linh lê bê đê séo xắc các bạn ơi khi các bạn thấy một cô gái quyết_định cắt tóc thì ko có_nghĩa_là cô đang buồn hay thất_tình đâu fmà là cô gái ấy bị nứng lồz bị một thế_lực thù_địch ma xui quỷ khiến nào đó làm cô nhồng lên đi cắt tóc và giờ cô đang căm vãi cả lồn khi nghĩ đến những lời đùa_cợt của mấy cdn đĩ bạn thân lớp vì mái_tóc cô giờ_đây nhìn đéo khác gì cái đb cuốn giẻ mai cz khi cô phải mang theo sún

In [ ]:
# in ra thống kê về lỗi vừa tạo
print('Tổng số lỗi:          ', total)
print('SL câu được tạo lỗi:  ', cho)
print('Tỉ lệ tạo lỗi:        ', total/cho , 'lỗi/câu')

Tổng số lỗi:           1038
SL câu được tạo lỗi:   356
Tỉ lệ tạo lỗi:         2.9157303370786516 lỗi/câu


In [ ]:
# in mẫu ra các lỗi
for i in range(len(test_y)):
  if list(test_y[i]) != [1,0,0]:
    print(test_x_mt[i])

gần đất xa trời rồi để cho bà_con thương thằnt cxó mmày chết sớm dđi
thôi cron chó ncu câm mồm với cty con sản_xuất điện nhưng đéo phải là mình nó chưa kể nhập_khẩu điện và nó vẫn là trung_gian giữa các nguồn khác tới ng dân dù ít ngta vẫn cần minh_bạch chứ đéo phải lấy về quản_lý hết bán ra mà dân cứ ngu_ngơ như con chó mqày đấy
lê bằng toàn đr khóc thuê hộ chel thôi đàn_ông fmà khóc nhưr đàn_bà
xe vô cảng ra khỏi cảng đã chửi lộn hết ngưi chứ không thrì nha anh anh thông_cảm bà cha ai thông_cảm coho ops ai thông_cảm cho tài_xế cont nữa rớt lại luôn không_thể_nào cứu_chữa
bồng_bột linh lê bê đê séo xắc các bạn ơi khi các bạn thấy một cô gái quyết_định cắt tóc thì ko có_nghĩa_là cô đang buồn hay thất_tình đâu fmà là cô gái ấy bị nứng lồz bị một thế_lực thù_địch ma xui quỷ khiến nào đó làm cô nhồng lên đi cắt tóc và giờ cô đang căm vãi cả lồn khi nghĩ đến những lời đùa_cợt của mấy cdn đĩ bạn thân lớp vì mái_tóc cô giờ_đây nhìn đéo khác gì cái đb cuốn giẻ mai cz khi cô phải mang theo sún

Lưu file test xuống

In [ ]:
dump(test_x_mt, 'test_x.dump', '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/TN1/')

# Test mô hình với các phương pháp khác nhau

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras import backend as K
from keras.models import model_from_json
from keras.models import load_model
from sklearn.metrics import classification_report as report
import pickle

In [ ]:
dir_folder = '/content/drive/My Drive/Colab Notebooks/dump_hatespeech/'
dir_folder_fix = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/save_data/symspell/'
# hàm load file lên
def dump(variable, file_name, dir_folder):
  f = open(dir_folder + file_name, 'wb')
  pickle.dump(variable, f)
  f.close()

def load(file_name, dir_folder):
  f = open(dir_folder + file_name, 'rb')
  variable = pickle.load(f)
  f.close()
  return variable

Khởi tạo mô hình

In [ ]:
EMBEDDING_FILE =  '/content/drive/My Drive/Colab Notebooks/Hate Speech Detection/Data/cc.vi.300.vec'
train_path =      '/content/drive/My Drive/Colab Notebooks/Hate Speech Detection/Data/Train.csv'
test_path =       '/content/drive/My Drive/Colab Notebooks/Hate Speech Detection/Data/Test.csv'
weight_path =     '/content/drive/My Drive/Colab Notebooks/dump_hatespeech/model.h5'
json_path =       '/content/drive/My Drive/Colab Notebooks/dump_hatespeech/model_num_bc.json'
result_path =     '/content/drive/My Drive/Colab Notebooks/Hate Speech Detection/Result/Result.csv'


# train = pd.read_csv(train_path).fillna(" ")
# test = pd.read_csv(test_path).fillna(" ")


max_features=3000
maxlen=150
embed_size=300

# train['free_text'].fillna(' ')
# test['free_text'].fillna(' ')

# train_y = train[['CLEAN', 'OFFENSIVE', 'HATE']].values

# train_x = train['free_text'].str.lower()

# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, train_size = 0.8)
###### lưu hết 4 biến trên xuống
train_x = load('train_x.dump', dir_folder)
test_x = load('test_x.dump' , dir_folder)
train_y = load('train_y.dump', dir_folder)
test_y = load('test_y.dump' , dir_folder)

# train_x = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/NB/gg_correct.txt').read().split('\n')
# test_x = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/log_result/NB/gg_test_out.txt').read().split('\n')

# test_x = test['free_text'].str.lower()

# Vectorize text + Prepare Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)


test_x = sequence.pad_sequences(test_x, maxlen=maxlen)

print("create vector")
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(3, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


create vector


Load bộ trọng số đã được train với dữ liệu bình thường trước đó lên

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/TN1/save_model/model_9.h5')

Tiến hành đánh giá mô hình và lưu kết quả xuống

In [ ]:
from sklearn.metrics import classification_report

method = ['origin', 'spelling_mistake','skip-gram', 'cbow', 'fasttext_fb', 'fasttext_vne', 'symspell', 'google']
report = ''
for i in method:
  test_x_mt_sent = list(load(i + '_test.dump', '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/TN1/correct/'))
  test_x_mt = tokenizer.texts_to_sequences(test_x_mt_sent)
  test_x_mt = sequence.pad_sequences(test_x_mt, maxlen=maxlen)
  predictions = model.predict(test_x_mt, batch_size=1024, verbose=1)
  test_pre = [] 
  for j in predictions:
    max = 0
    imax = 0
    for index, y in enumerate(j):
      if y > max:
        max = y 
        imax = index
    t = [0,0,0]
    t[imax] = 1
    test_pre.append(t)
  s = ''
  temp = list(np.argmax(test_pre, axis=1))
  for j in range(len(temp)):
    s = s + str(temp[j]) + '\n' + test_x_mt_sent[j] + '\n------------------\n'
  open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/TN1/correct/result/'+ i + '.txt', 'w').write(s)
  report = report + "Method:\t" + i.upper() + '\n' + classification_report(np.argmax(test_y, axis=1), np.argmax(test_pre, axis=1), target_names=['CLEAN', 'OFFENSIVE', 'HATE']) + '\n--------------------------------------\n\n'

4/4 [==============================] - 23s 6s/step


Ghi kết quả thành file

In [ ]:
open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/TN1/correct/report.txt', 'w').write(report)

3512